In [14]:
#run once - small sets of vectors for development (low-D, high minimum frequency threshold so relatively few concepts)
!wget https://www.dropbox.com/s/3f5qg9nfop2l45e/semanticvectors.bin
!wget https://www.dropbox.com/s/r8r8ffw9rnjmmo0/permutationvectors.bin
!wget https://www.dropbox.com/s/nf28cvp1ocvyh1s/elementalvectors.bin

--2019-07-01 15:47:01--  https://www.dropbox.com/s/3f5qg9nfop2l45e/semanticvectors.bin
Resolving www.dropbox.com... 162.125.1.1
Connecting to www.dropbox.com|162.125.1.1|:443... connected.


HTTP request sent, awaiting response... 

301 Moved Permanently
Location: /s/raw/3f5qg9nfop2l45e/semanticvectors.bin [following]
--2019-07-01 15:47:01--  https://www.dropbox.com/s/raw/3f5qg9nfop2l45e/semanticvectors.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 

302 Found
Location: https://uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com/cd/0/inline/Aj6GLa_9QVcZGTGQRv2xhLJ4Y8cIoTgjoTlISVwJmWQi-G3j2-VQaYiTlW1hDzcdMvx1N0ICIqu66QOf2Sqt8YpLC4wWUmDJ5MMn3zLnh9wqsw/file# [following]
--2019-07-01 15:47:01--  https://uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com/cd/0/inline/Aj6GLa_9QVcZGTGQRv2xhLJ4Y8cIoTgjoTlISVwJmWQi-G3j2-VQaYiTlW1hDzcdMvx1N0ICIqu66QOf2Sqt8YpLC4wWUmDJ5MMn3zLnh9wqsw/file
Resolving uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com... 162.125.1.6
Connecting to uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 

302 FOUND
Location: /cd/0/inline2/Aj7pmKkc0QtxfGOvnWz3ilqbwTyvKD0tDsDcosd4KU_hDYJ0ngriPQq6ggl1wkvl8aFtr513_soztqSvr0jAKuJTvCx-C_1ekvOi_p_G3_WJbH15VdqtLcUbjywM9WvVGy_Yvio4vrJkJcGQXUM6vu2WZ3X_98MpFi7GSPJO70IfgZVkyq6hnikUBaWTJcMBHaA4YepeDSXNHn7U8hMKdH55bwmpwaEUXCQj16kpDl9r3hYk1mc52GP9KJGmMknWZ6PeAf2_T24vZE-NIqIR-0LkCt6GqAXe5xi9KGK1ok3FQ0GoUuUnJLVym75J8KBtrovoxaQNi_xhX1S6VW7r0L8g/file [following]
--2019-07-01 15:47:02--  https://uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com/cd/0/inline2/Aj7pmKkc0QtxfGOvnWz3ilqbwTyvKD0tDsDcosd4KU_hDYJ0ngriPQq6ggl1wkvl8aFtr513_soztqSvr0jAKuJTvCx-C_1ekvOi_p_G3_WJbH15VdqtLcUbjywM9WvVGy_Yvio4vrJkJcGQXUM6vu2WZ3X_98MpFi7GSPJO70IfgZVkyq6hnikUBaWTJcMBHaA4YepeDSXNHn7U8hMKdH55bwmpwaEUXCQj16kpDl9r3hYk1mc52GP9KJGmMknWZ6PeAf2_T24vZE-NIqIR-0LkCt6GqAXe5xi9KGK1ok3FQ0GoUuUnJLVym75J8KBtrovoxaQNi_xhX1S6VW7r0L8g/file
Reusing existing connection to uc6c457017824c32dbc8f954f691.dl.dropboxusercontent.com:443.
HTTP request sent, awaiting response... 

...

206 PARTIAL CONTENT
Length: 27530388 (26M), 9086784 (8.7M) remaining [application/octet-stream]
Saving to: ‘elementalvectors.bin’

elementalvectors.bi  66%[+++++++++++++       ]  17.59M  --.-KB/s               

elementalvectors.bi  98%[+++++++++++++=====> ]  25.78M  --.-KB/s    in 0.1s    

2019-07-01 15:47:30 (64.6 MB/s) - Read error at byte 27027858/27530388 (error:1408F119:SSL routines:SSL3_GET_RECORD:decryption failed or bad record mac). Retrying.



--2019-07-01 15:47:36--  (try: 7)  https://uc56cd63226f3e383bd782fae7ef.dl.dropboxusercontent.com/cd/0/inline2/Aj7cvszasKh1DPIyaoNY0BKUXy-3ItWnjpZge6mfsTLyRXOtjuGVOvh5tTfpWMiIYHmgEdTdlzSf9Afiv7iJp8tPjuBGo1gDfzkVx7sgqNA0IlamMfyumC98oMM_jqFfZkopMCUkTQ8yLaZumwocUOh3yEAD0JVG2Wentd2a5f9LVt3wBzLs02qqH3XvdjuCRGLa0kyVFOmlvY5ozvx01HNNfzFyRzKIif88z1WcfIU06wbEn9y2DOFin-BtmCodOG5vLwIVqROgdIrTDi0KWspvbhxDg3BP58E446ngVpJGQXrVTzvq5SQCfKlHGQI1MtrtoPzdxBi2Txruhx09-KbQ/file
Connecting to uc56cd63226f3e383bd782fae7ef.dl.dropboxusercontent.com|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 

206 PARTIAL CONTENT
Length: 27530388 (26M), 502530 (491K) remaining [application/octet-stream]
Saving to: ‘elementalvectors.bin’

elementalvectors.bi  98%[+++++++++++++++++++ ]  25.78M  --.-KB/s               

elementalvectors.bi 100%[+++++++++++++++++++>]  26.25M  --.-KB/s    in 0.07s   

2019-07-01 15:47:36 (6.47 MB/s) - ‘elementalvectors.bin’ saved [27530388/27530388]



In [11]:
import copy
import permutations.constants as c
import permutations.sparse_permutations as sp
import permutations.semvec_utils as sv

In [12]:
#run once 
semanticvectors = sv.readfile('semanticvectors.bin')
elementalvectors = sv.readfile('elementalvectors.bin')
permutation_cache = sv.readfile('permutationvectors.bin')


31
100   REAL


y 156
((3z)-n-(3-chlorophenyl)-3-((3,5-dimethyl-4-((4-methylpiperazin-1-yl)carbonyl)-1h-pyrrol-2-yl)methylene)-n-methyl-2-oxo-2,3-dihydro-1h-indole-5-sulfonamide)
skipped 0


31
100   REAL


y 156
((3z)-n-(3-chlorophenyl)-3-((3,5-dimethyl-4-((4-methylpiperazin-1-yl)carbonyl)-1h-pyrrol-2-yl)methylene)-n-methyl-2-oxo-2,3-dihydro-1h-indole-5-sulfonamide)
skipped 0


38
100   PERMUTATION
skipped 0


In [22]:
#nearest neighbor by term or by vector
print(sv.get_k_vec_neighbors(semanticvectors,'docetaxel',10))
print(sv.get_k_neighbors(semanticvectors,sv.getvector(semanticvectors,'docetaxel'),10))

['docetaxel', 'gemcitabine', 'etoposide', 'paclitaxel', 'carboplatin', 'erlotinib', 'gefitinib', 'capecitabine', 'cetuximab', 'fluorouracil']
['docetaxel', 'gemcitabine', 'etoposide', 'paclitaxel', 'carboplatin', 'erlotinib', 'gefitinib', 'capecitabine', 'cetuximab', 'fluorouracil']


In [23]:
#single predicate search
semvec=copy.copy(sv.getvector(semanticvectors,'prozac'))
permvec=sp.permute_vector(sv.getvector(permutation_cache,'_ISA'),semvec)
results=sv.get_k_neighbors(elementalvectors, permvec, 10)
print(results)

['dopamine_antagonists', 'antidepressive_agents', 'monoamine_oxidase_inhibitors', 'serotonin_uptake_inhibitors', 'reuptake_inhibitors', 'antipsychotic_agents', 'selective_serotonin_re-uptake_inhibitor', 'antidepressive_agents,_tricyclic', 'atypical_antipsychotic', 'serotonin_agents']


In [21]:
#double predicate search - not sure why plants and cattle show up, but the others make sense
semvec=copy.copy(sv.getvector(semanticvectors,'prozac'))
permvec=sp.permute_vector(sv.getvector(permutation_cache,'_ISA'),semvec)
permvec=sp.permute_vector(sv.getvector(permutation_cache,'TREATS-INV'),permvec)
results=sv.get_k_neighbors(semanticvectors, permvec, 10)
print(results)

['plants', 'obsessive-compulsive_disorder', 'phobia,_social', 'depression,_bipolar', 'schizoaffective_disorder', 'sleeplessness', 'relationships', 'cattle', 'chronic_schizophrenia', 'country']
